# LangSmith聊天数据集

本笔记本演示了一种简单的方法来加载LangSmith聊天数据集并在该数据上微调模型。
该过程非常简单，包含3个步骤。

1. 创建聊天数据集。
2. 使用LangSmithDatasetChatLoader加载示例。
3. 微调您的模型。

然后您可以在LangChain应用中使用微调后的模型。

在开始之前，让我们先安装必要的依赖项。

## 前提条件

确保您已安装langchain >= 0.0.311，并已在环境中配置了LangSmith API密钥。

In [ ]:
%pip install --upgrade --quiet  langchain langchain-openai

In [1]:
import os
import uuid

uid = uuid.uuid4().hex[:6]
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = "YOUR API KEY"

## 1. 选择数据集

本笔记本直接基于选择的运行结果微调模型。您通常会从跟踪的运行中筛选这些结果。您可以在[文档](https://docs.smith.langchain.com/evaluation/concepts#datasets)中了解更多关于LangSmith数据集的信息。

为了本教程的目的，我们将在此上传一个现有的数据集供您使用。

In [2]:
from langsmith.client import Client

client = Client()

In [3]:
import requests

url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/integrations/chat_loaders/example_data/langsmith_chat_dataset.json"
response = requests.get(url)
response.raise_for_status()
data = response.json()

In [4]:
dataset_name = f"Extraction Fine-tuning Dataset {uid}"
ds = client.create_dataset(dataset_name=dataset_name, data_type="chat")

In [5]:
_ = client.create_examples(
    inputs=[e["inputs"] for e in data],
    outputs=[e["outputs"] for e in data],
    dataset_id=ds.id,
)

## 2. 准备数据
现在我们可以创建一个LangSmithRunChatLoader实例，并使用其lazy_load()方法加载聊天会话。

In [6]:
from langchain_community.chat_loaders.langsmith import LangSmithDatasetChatLoader

loader = LangSmithDatasetChatLoader(dataset_name=dataset_name)

chat_sessions = loader.lazy_load()

#### 加载聊天会话后，将它们转换为适合微调的格式。

In [7]:
from langchain_community.adapters.openai import convert_messages_for_finetuning

training_data = convert_messages_for_finetuning(chat_sessions)

## 3. 微调模型
现在，使用OpenAI库启动微调过程。

In [8]:
import json
import time
from io import BytesIO

import openai

my_file = BytesIO()
for dialog in training_data:
    my_file.write((json.dumps({"messages": dialog}) + "\n").encode("utf-8"))

my_file.seek(0)
training_file = openai.files.create(file=my_file, purpose="fine-tune")

job = openai.fine_tuning.jobs.create(
    training_file=training_file.id,
    model="gpt-3.5-turbo",
)

# Wait for the fine-tuning to complete (this may take some time)
status = openai.fine_tuning.jobs.retrieve(job.id).status
start_time = time.time()
while status != "succeeded":
    print(f"Status=[{status}]... {time.time() - start_time:.2f}s", end="\r", flush=True)
    time.sleep(5)
    status = openai.fine_tuning.jobs.retrieve(job.id).status

# Now your model is fine-tuned!

## 4. 在LangChain中使用

微调完成后，在LangChain应用中使用结果模型ID与ChatOpenAI模型类。

In [10]:
# Get the fine-tuned model ID
job = openai.fine_tuning.jobs.retrieve(job.id)
model_id = job.fine_tuned_model

# Use the fine-tuned model in LangChain
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model=model_id,
    temperature=1,
)

In [11]:
model.invoke("There were three ravens sat on a tree.")

AIMessage(content='[{"s": "There were three ravens", "object": "tree", "relation": "sat on"}, {"s": "three ravens", "object": "a tree", "relation": "sat on"}]')

现在您已成功使用LangSmith LLM运行的数据微调了一个模型！